In [169]:
import pandas as pd
from datetime import datetime
import numpy as np
import warnings

In [3]:
#get the data
data = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2019_totals.html')[0]
data.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Álex Abrines,SG,25,OKC,31,2,588,56,157,...,.923,5,43,48,20,17,6,14,53,165
1,2,Quincy Acy,PF,28,PHO,10,0,123,4,18,...,.700,3,22,25,8,1,4,4,24,17
2,3,Jaylen Adams,PG,22,ATL,34,1,428,38,110,...,.778,11,49,60,65,14,5,28,45,108
3,4,Steven Adams,C,25,OKC,80,80,2669,481,809,...,.500,391,369,760,124,117,76,135,204,1108
4,5,Bam Adebayo,C,21,MIA,82,28,1913,280,486,...,.735,165,432,597,184,71,65,121,203,729


In [6]:
#check to make sure the data loaded in correctly
data.describe()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,734,734,734,734,734,734,734,734,734,734,...,691,734,734,734,734,734,734,734,734,734
unique,531,531,12,23,32,83,81,586,331,469,...,273,154,308,351,246,114,97,184,215,490
top,Rk,Player,SG,23,TOT,G,0,MP,FG,FGA,...,FT%,0,DRB,TRB,0,1,0,0,0,PTS
freq,26,26,176,75,86,26,199,26,26,26,...,26,42,26,26,27,45,90,36,26,26


In [8]:
#check for team names
data.Tm.unique()

array(['OKC', 'PHO', 'ATL', 'MIA', 'CLE', 'DEN', 'SAS', 'CHI', 'UTA',
       'BRK', 'NYK', 'POR', 'MEM', 'TOT', 'IND', 'MIL', 'DAL', 'HOU',
       'Tm', 'TOR', 'WAS', 'ORL', 'CHO', 'SAC', 'LAL', 'MIN', 'BOS',
       'GSW', 'NOP', 'LAC', 'PHI', 'DET'], dtype=object)

In [10]:
#get the proper team names
rockets = data.loc[data.Tm == "HOU"]
bucks = data.loc[data.Tm == 'MIL']
rockets.shape, bucks.shape

((23, 30), (24, 30))

In [15]:
#join the two teams together
rnb = pd.concat([rockets, bucks], axis=0, join='outer', ignore_index = True)
rnb.shape

(47, 30)

In [16]:
#save the files as jsons
rockets.to_json('assets/rockets.json')
bucks.to_json('assets/bucks.json')
rnb.to_json('assets/rnb.json')

In [17]:
data.to_json('assets/nba.json')

In [8]:
#get the scores by month
months = ['october', 'november', 'december', 
          'january', 'february', 'march',
         'april', 'may']

#put all the scores into a dataframe
for month in months:
    if month == 'october':
        month_data = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2019_games-'+month+ '.html')[0]
    else:
        temp = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2019_games-'+month+ '.html')[0]
        month_data = pd.concat([month_data, temp], axis=0, join='outer', ignore_index = True)


(1308, 10)

In [35]:
#only keep the regular season data, playoffs start at index 1230
regular_season = month_data[:1230]

#helper method to clean
def date_formatter(elt):
    cleaned = elt.replace(',','')
    return datetime.strptime(cleaned, '%a %b %d %Y')

#hehe sorry filter the warning for aesthetic
warnings.filterwarnings('ignore')
regular_season['Date'] = regular_season['Date'].apply(lambda x: date_formatter(x))
#check to make sure method worked
regular_season.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,2018-10-16,8:00p,Philadelphia 76ers,87,Boston Celtics,105,Box Score,NaN,18624,NaN
1,2018-10-16,10:30p,Oklahoma City Thunder,100,Golden State Warriors,108,Box Score,NaN,19596,NaN
2,2018-10-17,7:00p,Milwaukee Bucks,113,Charlotte Hornets,112,Box Score,NaN,17889,NaN
3,2018-10-17,7:00p,Brooklyn Nets,100,Detroit Pistons,103,Box Score,NaN,20332,NaN
4,2018-10-17,7:00p,Memphis Grizzlies,83,Indiana Pacers,111,Box Score,NaN,17923,NaN


In [142]:
#helper method to find the winner of the game
def winner(row):
    if row['PTS'] > row['PTS.1']:
        return row['Visitor/Neutral']
    else:
        return row['Home/Neutral']

#apply helper to get the winner of each game
regular_season['Winner'] = regular_season.apply(winner, axis = 1)
regular_season.head()

#for our purposes, we only need to know the date and the winner
winners_only = regular_season[['Date','Winner']]
winners_only.head()

,Date,Winner
0,2018-10-16,Boston Celtics
1,2018-10-16,Golden State Warriors
2,2018-10-17,Milwaukee Bucks
3,2018-10-17,Detroit Pistons
4,2018-10-17,Indiana Pacers


In [151]:
#find games where Houston wins
houston = winners_only['Winner'] == 'Houston Rockets'
milwaukee =  winners_only['Winner'] == 'Milwaukee Bucks'

winners_only['HOU_wins'] = houston.apply(lambda x: int(x == True))
winners_only['MIL_wins'] = milwaukee.apply(lambda x: int(x == True))

def hou_or_mil(elt):
    return elt == 'Houston Rockets' or elt == 'Milwaukee Bucks'

hou_mil = winners_only['Winner'].apply(hou_or_mil)
hou_mil_wins = winners_only.loc[hou_mil].reset_index(drop = True)
hou_mil_wins.head()

,Date,Winner,HOU_wins,MIL_wins
0,2018-10-17,Milwaukee Bucks,0,1
1,2018-10-19,Milwaukee Bucks,0,1
2,2018-10-20,Houston Rockets,1,0
3,2018-10-22,Milwaukee Bucks,0,1
4,2018-10-24,Milwaukee Bucks,0,1


In [176]:
hou_wins = winners_only.loc[houston].reset_index(drop = True).drop('MIL_wins', axis = 1)
mil_wins = winners_only.loc[milwaukee].reset_index(drop = True).drop('HOU_wins', axis = 1)

def win_ct(row):
    return row.name + 1

hou_wins['num_wins'] = hou_wins.apply(win_ct, axis = 1)
mil_wins['num_wins'] = mil_wins.apply(win_ct, axis = 1)

cleaned_wins = pd.concat([hou_wins, mil_wins], axis=0, join='outer', ignore_index = True).drop(['HOU_wins', 'MIL_wins'], axis = 1)
cleaned_wins.to_json('hou_mil_wins.json')

# temp = hou_mil_wins.merge(hou_wins, how = 'outer', left_on = ['Date','Winner'], right_on= ['Date', 'Winner'])
# temp_hou_mil = temp.merge(mil_wins, how = 'outer', left_on = ['Date','Winner'], right_on= ['Date', 'Winner'])
# temp_hou_mil = temp_hou_mil.replace(np.NaN, 0)
# temp_hou_mil['num_wins']